In [1]:
def get_dominant_emotion(emotions):
    emotion_counts = {}
    if len(emotions) == 0:
        dominant_emotion = ""
        return dominant_emotion 
    for emotion in emotions:
        if emotion not in emotion_counts:
            emotion_counts[emotion] = 0
        emotion_counts[emotion] += 1
    dominant_emotion = max(emotion_counts, key=emotion_counts.get)
    return dominant_emotion



def get_triangle_area(rectangle):
    # Récupérer les coordonnées du rectangle
    x1, y1, x2, y2 = rectangle

    # Calculer l'aire du rectangle comme une approximation de l'aire du triangle
    triangle_area = (x2 - x1) * (y2 - y1)

    return triangle_area

def find_largest_triangle(unique_face):
    # Trouver l'indice du rectangle le plus grand
    index_plus_grand = max(range(len(unique_face)), key=lambda i: get_triangle_area(unique_face[i]))

    return index_plus_grand

Sans la position

In [4]:
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter
from keras.models import load_model
import os
import csv

image_folder = r'Multi_Label_dataset\Images'
output_csv = 'nbvisage_emotion.csv'

classCascade = cv2.CascadeClassifier("ressources\haarcascade_frontalface_alt2.xml")
profile_cascade = cv2.CascadeClassifier("ressources\haarcascade_profileface.xml")
emotion_model = load_model("ressources\FER_model.h5")



# Obtenez la liste des fichiers d'images dans le dossier
image_files = os.listdir(image_folder)

# Ouvrir le fichier CSV en mode écriture
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'nb_visages', 'emotions'])  # Écrire l'en-tête des colonnes

    # Parcourir les fichiers d'images
    for image_file in image_files:
        # Obtenez le chemin complet de l'image
        image_path = os.path.join(image_folder, image_file)

        # Réinitialiser les listes pour les informations de l'image actuelle
        tab_face = []
        
        emovisages = []   

        # Charger l'image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Effectuer la détection des visages
        faces = classCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages détectés à la liste
        for (x, y, w, h) in faces:
            tab_face.append((x, y, x+w, y+h))

        # Effectuer la détection des visages de profil
        profiles = profile_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil détectés à la liste
        for (x, y, w, h) in profiles:
            tab_face.append((x, y, x+w, y+h))

        # Inverser l'image en niveaux de gris
        width = gray.shape[1]
        gray2 = cv2.flip(gray, 1)

        # Effectuer la détection des visages de profil inversés
        profiles_inv = profile_cascade.detectMultiScale(
            gray2,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil inversés détectés à la liste
        for (x, y, w, h) in profiles_inv:
            tab_face.append((width - (x + w), y, width - x, y + h))
        
        
        # Vérifier s'il y a des visages détectés
        if len(tab_face) != 0:
            
            emovisages = []            
            # Conversion de la liste en tableau numpy pour l'entrée de DBSCAN
            face_array = np.array(tab_face)

            # Paramètres de DBSCAN
            eps = 50  # Rayon de recherche pour la densité
            min_samples = 2  # Nombre minimum d'échantillons pour former un cluster

            # Application de DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(face_array)

            # Création de la liste des visages uniques après regroupement
            unique_faces = []
            unique_labels = set(labels)
            for label in unique_labels:
                indices = np.where(labels == label)[0]
                unique_faces.append(face_array[indices[0]])

            # Obtenir les émotions pour chaque visage unique
            for (x1, y1, x2, y2) in unique_faces:
                face_gray = gray[y1:y2, x1:x2]
                face_resized = cv2.resize(face_gray, (48, 48))
                face_normalized = face_resized.astype("float") / 255.0
                face_array = np.expand_dims(face_normalized, axis=0)

                prediction = emotion_model.predict(face_array)[0]
                max_index = np.argmax(prediction)
                emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
                emotion = emotions[max_index]
                emovisages.append(emotion)

        if len(emovisages) == 1:
            writer.writerow([os.path.splitext(image_file)[0], len(emovisages), emovisages[0]] )
        elif len(emovisages) == 2:
            writer.writerow([os.path.splitext(image_file)[0], len(emovisages), emovisages[find_largest_triangle(unique_faces)]] )

        else :
            writer.writerow([os.path.splitext(image_file)[0], len(emovisages), get_dominant_emotion(emovisages)] )



        


print("Terminé ! Les informations ont été enregistrées dans le fichier CSV.")


1/1 [==============================] - 0s 41ms/step
Terminé ! Les informations ont été enregistrées dans le fichier CSV.


Avec position

In [3]:
import cv2
import os
import csv
import numpy as np
from sklearn.cluster import DBSCAN
from keras.models import load_model

def get_dominant_emotion(emotions):
    emotion_counts = {}
    if len(emotions) == 0:
        dominant_emotion = "Neutral"
        return dominant_emotion 
    for emotion in emotions:
        if emotion not in emotion_counts:
            emotion_counts[emotion] = 0
        emotion_counts[emotion] += 1
    dominant_emotion = max(emotion_counts, key=emotion_counts.get)
    return dominant_emotion

def get_triangle_area(rectangle):
    # Récupérer les coordonnées du rectangle
    x1, y1, x2, y2 = rectangle
    # Calculer l'aire du rectangle comme une approximation de l'aire du triangle
    triangle_area = (x2 - x1) * (y2 - y1)
    return triangle_area

def find_largest_triangle(unique_faces):
    # Trouver l'indice du rectangle le plus grand
    index_plus_grand = max(range(len(unique_faces)), key=lambda i: get_triangle_area(unique_faces[i]))

    return index_plus_grand

# Chargement des classificateurs cascade pour la détection des visages
classCascade = cv2.CascadeClassifier("ressources/haarcascade_frontalface_alt2.xml")
profile_cascade = cv2.CascadeClassifier("ressources/haarcascade_profileface.xml")
emotion_model = load_model("ressources/FER_model.h5")

# Dossier contenant les images
image_folder = "../../Multi_Label_dataset/Images"
output_csv = "nb_pos_emo_visage.csv"

# Ouvrir le fichier CSV en mode écriture
with open(output_csv, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'nb_visages', 'position_visage', 'emotion1', 'emotion2'])

    # Parcourir les fichiers d'images
    for image_file in os.listdir(image_folder):
        # Obtenez le chemin complet de l'image
        image_path = os.path.join(image_folder, image_file)

        # Réinitialiser les listes pour les informations de l'image actuelle
        tab_face = []
        emovisages = []

        # Charger l'image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Effectuer la détection des visages
        faces = classCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages détectés à la liste
        for (x, y, w, h) in faces:
            tab_face.append((x, y, x+w, y+h))

        # Effectuer la détection des visages de profil
        profiles = profile_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil détectés à la liste
        for (x, y, w, h) in profiles:
            tab_face.append((x, y, x+w, y+h))

        # Inverser l'image en niveaux de gris
        width = gray.shape[1]
        gray2 = cv2.flip(gray, 1)

        # Effectuer la détection des visages de profil inversés
        profiles_inv = profile_cascade.detectMultiScale(
            gray2,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil inversés détectés à la liste
        for (x, y, w, h) in profiles_inv:
            tab_face.append((width - (x + w), y, width - x, y + h))

        # Vérifier s'il y a des visages détectés
        if len(tab_face) != 0:
            emovisages = []

            # Conversion de la liste en tableau numpy pour l'entrée de DBSCAN
            face_array = np.array(tab_face)

            # Paramètres de DBSCAN
            eps = 50  # Rayon de recherche pour la densité
            min_samples = 2  # Nombre minimum d'échantillons pour former un cluster

            # Application de DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(face_array)

            # Création de la liste des visages uniques après regroupement
            unique_faces = []
            unique_labels = set(labels)
            for label in unique_labels:
                indices = np.where(labels == label)[0]
                unique_faces.append(face_array[indices[0]])

            # Obtenir les émotions pour chaque visage unique
            for (x1, y1, x2, y2) in unique_faces:
                face_gray = gray[y1:y2, x1:x2]
                face_resized = cv2.resize(face_gray, (48, 48))
                face_normalized = face_resized.astype("float") / 255.0
                face_array = np.expand_dims(face_normalized, axis=0)

                prediction = emotion_model.predict(face_array)[0]
                max_index = np.argmax(prediction)
                emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
                emotion = emotions[max_index]
                emovisages.append(emotion)

            if len(emovisages) == 0:
                writer.writerow([os.path.splitext(image_file)[0], "Neutral", "Neutral", "Neutral", "Neutral"])
            elif len(emovisages) == 1:
                face_position = "Haut" if (y1 + y2) / 2 < gray.shape[0] / 2 else "Bas"
                writer.writerow([os.path.splitext(image_file)[0], 1, face_position, emovisages[0], "Neutral"])
            else:
                face_position = "Haut" if (y1 + y2) / 2 < gray.shape[0] / 2 else "Bas"
                largest_triangle_index = find_largest_triangle(unique_faces)
                emotion1 = emovisages[largest_triangle_index]
                if largest_triangle_index == 0:
                    emotion2 = emovisages[1] if len(emovisages) > 1 else "Neutral"
                else:
                    emotion2 = emovisages[0]
                writer.writerow([os.path.splitext(image_file)[0], len(emovisages), face_position, emotion1, emotion2])
        else:
            writer.writerow([os.path.splitext(image_file)[0],"Neutral", "Neutral", "Neutral", "Neutral"])

print("Terminé ! Les informations ont été enregistrées dans le fichier CSV.")


1/1 [==============================] - 0s 257ms/step
Terminé ! Les informations ont été enregistrées dans le fichier CSV.


In [6]:
import csv

input_csv = "nb_pos_emo_visage.csv"
output_csv = "nb_pos_emo_visage_num.csv"

# Mapping des valeurs textuelles aux valeurs numériques
position_mapping = {"Neutral": 0, "Haut": 1, "Bas": 2}
emotion_mapping = {"Neutral": 0, "Sad": 1, "Happy": 2, "Surprise": 3, "Angry": 4, "Fear": 5}

# Ouvrir le fichier CSV d'entrée en mode lecture et le fichier CSV de sortie en mode écriture
with open(input_csv, "r") as csvfile_in, open(output_csv, "w", newline='') as csvfile_out:
    reader = csv.reader(csvfile_in)
    writer = csv.writer(csvfile_out)

    # Copier l'en-tête du fichier CSV d'entrée dans le fichier CSV de sortie
    header = next(reader)
    writer.writerow(header)

    # Parcourir les lignes du fichier CSV d'entrée
    for row in reader:
        # Modifier les valeurs selon les mappings définis
        position = position_mapping.get(row[2], row[2])
        emotion1 = emotion_mapping.get(row[3], row[3])
        emotion2 = emotion_mapping.get(row[4], row[4])

        # Écrire la ligne modifiée dans le fichier CSV de sortie
        writer.writerow([row[0], int(row[1]) if row[1] != "Neutral" else 0, position, emotion1, emotion2])

print("Le fichier CSV a été modifié avec succès.")


Le fichier CSV a été modifié avec succès.


Avec position rectifié

In [5]:
import cv2
import os
import csv
import numpy as np
from sklearn.cluster import DBSCAN
from keras.models import load_model



# Chargement des classificateurs cascade pour la détection des visages
classCascade = cv2.CascadeClassifier("ressources/haarcascade_frontalface_alt2.xml")
profile_cascade = cv2.CascadeClassifier("ressources/haarcascade_profileface.xml")
emotion_model = load_model("ressources/FER_model.h5")

# Dossier contenant les images
image_folder = "../../Multi_Label_dataset/Images"
output_csv = "new_nb_pos_emo_visage.csv"

# Ouvrir le fichier CSV en mode écriture
with open(output_csv, "w", newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'nb_visages', 'position_visage', 'emotion1', 'emotion2'])

    # Parcourir les fichiers d'images
    for image_file in os.listdir(image_folder):
        # Obtenez le chemin complet de l'image
        image_path = os.path.join(image_folder, image_file)

        # Réinitialiser les listes pour les informations de l'image actuelle
        tab_face = []
        emovisages = []

        # Charger l'image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Effectuer la détection des visages
        faces = classCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages détectés à la liste
        for (x, y, w, h) in faces:
            tab_face.append((x, y, x+w, y+h))

        # Effectuer la détection des visages de profil
        profiles = profile_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil détectés à la liste
        for (x, y, w, h) in profiles:
            tab_face.append((x, y, x+w, y+h))

        # Inverser l'image en niveaux de gris
        width = gray.shape[1]
        gray2 = cv2.flip(gray, 1)

        # Effectuer la détection des visages de profil inversés
        profiles_inv = profile_cascade.detectMultiScale(
            gray2,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Ajouter les coordonnées des visages de profil inversés détectés à la liste
        for (x, y, w, h) in profiles_inv:
            tab_face.append((width - (x + w), y, width - x, y + h))

        # Vérifier s'il y a des visages détectés
        if len(tab_face) != 0:
            emovisages = []

            # Conversion de la liste en tableau numpy pour l'entrée de DBSCAN
            face_array = np.array(tab_face)

            # Paramètres de DBSCAN
            eps = 50  # Rayon de recherche pour la densité
            min_samples = 2  # Nombre minimum d'échantillons pour former un cluster

            # Application de DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(face_array)

            # Création de la liste des visages uniques après regroupement
            unique_faces = []
            unique_labels = set(labels)
            for label in unique_labels:
                indices = np.where(labels == label)[0]
                unique_faces.append(face_array[indices[0]])

            # Obtenir les émotions pour chaque visage unique
            for (x1, y1, x2, y2) in unique_faces:
                face_gray = gray[y1:y2, x1:x2]
                face_resized = cv2.resize(face_gray, (48, 48))
                face_normalized = face_resized.astype("float") / 255.0
                face_array = np.expand_dims(face_normalized, axis=0)

                prediction = emotion_model.predict(face_array)[0]
                max_index = np.argmax(prediction)
                emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
                emotion = emotions[max_index]
                emovisages.append(emotion)

            if len(emovisages) == 0:
                writer.writerow([os.path.splitext(image_file)[0], "Neutral", "Neutral", "Neutral", "Neutral"])
            elif len(emovisages) == 1:
                face_position = "Haut" if (y1 + y2) / 2 < gray.shape[0] / 2 else "Bas"
                writer.writerow([os.path.splitext(image_file)[0], 1, face_position, emovisages[0], "Neutral"])
            else:
                emotions = ["Neutral"] * min(2, len(emovisages))
                face_positions = ["Neutral"] * min(2, len(emovisages))

                visages_tries = sorted(unique_faces, key=lambda rect: (rect[2] - rect[0]) * (rect[3] - rect[1]), reverse=True)[:2]

                for i, visage in enumerate(visages_tries):
                    x1, y1, x2, y2 = visage
                    face_positions[i] = "Haut" if (y1 + y2) / 2 < gray.shape[0] / 2 else "Bas"
                    emotions[i] = emovisages[i]

                

                writer.writerow([os.path.splitext(image_file)[0], len(emovisages), face_positions[0], emotions[0], emotions[1]])
        else:
            writer.writerow([os.path.splitext(image_file)[0], "Neutral", "Neutral", "Neutral", "Neutral"])

print("Terminé ! Les informations ont été enregistrées dans le fichier CSV.")


1/1 [==============================] - 0s 71ms/step
Terminé ! Les informations ont été enregistrées dans le fichier CSV.


Numeriser les donnée

Numeriser les données modifiées

In [6]:
import csv

input_csv = "new_nb_pos_emo_visage.csv"
output_csv = "new_nb_pos_emo_visage_num.csv"

# Mapping des valeurs textuelles aux valeurs numériques
position_mapping = {"Neutral": 0, "Haut": 1, "Bas": 2}
emotion_mapping = {"Neutral": 0, "Sad": 1, "Happy": 2, "Surprise": 3, "Angry": 4, "Fear": 5}

# Ouvrir le fichier CSV d'entrée en mode lecture et le fichier CSV de sortie en mode écriture
with open(input_csv, "r") as csvfile_in, open(output_csv, "w", newline='') as csvfile_out:
    reader = csv.reader(csvfile_in)
    writer = csv.writer(csvfile_out)

    # Copier l'en-tête du fichier CSV d'entrée dans le fichier CSV de sortie
    header = next(reader)
    writer.writerow(header)

    # Parcourir les lignes du fichier CSV d'entrée
    for row in reader:
        # Modifier les valeurs selon les mappings définis
        position = position_mapping.get(row[2], row[2])
        emotion1 = emotion_mapping.get(row[3], row[3])
        emotion2 = emotion_mapping.get(row[4], row[4])

        # Écrire la ligne modifiée dans le fichier CSV de sortie
        writer.writerow([row[0], int(row[1]) if row[1] != "Neutral" else 0, position, emotion1, emotion2])

print("Le fichier CSV a été modifié avec succès.")


Le fichier CSV a été modifié avec succès.
